In [1]:
import ipywidgets as ipw
from IPython.display import display
import sys
import py

In [2]:
def capture(output_widget):
    def decorator(func):
        def decorated(*args, **kwargs):
            output_widget.clear_output()
            result = {}
            with output_widget:
                result = func(*args, **kwargs)
            return result
        return decorated
    return decorator

In [3]:
WORKCHAIN_PK = ipw.IntText(description='Workchain')

In [4]:
VERDI_BIN = ipw.Text(description='verdi bin', default='verdi')
#VERDI_BIN.value = py.path.local(sys.executable).dirpath().join('verdi').strpath
VERDI_BIN.value = 'verdi'

In [5]:
COMPUTER = ipw.HTML(description='Computer')

In [6]:
CHECK = ipw.Button(description='Check')
CHECK = ipw.Button(description='Check ')

In [7]:
STATE = ipw.HTML(description='State')

In [8]:
RUNTIME = ipw.HTML(description='Runtime')

In [9]:
FORMULA = ipw.HTML(description='Structure')

In [10]:
TREE = ipw.Output()

In [11]:
WORK_LIST = ipw.Output()

In [12]:
CALC_LIST = ipw.Output()

In [13]:
GET_INFO_ERR = ipw.Output()

In [14]:
BANDS = ipw.Output()

In [15]:
RESULTS = ipw.HTML(description='Results')

In [16]:
OUTPUT = ipw.HBox([RESULTS, BANDS])

In [17]:
def dict2html(in_dict):
    lines = ['{} = {}'.format(key, val) for key, val in in_dict.items()]
    return '<br/>'.join(lines)
        

In [18]:
def plot_mpl(all_data):
    from matplotlib import rc
    
    # Uncomment to change default font
    #rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    #rc('font', **{'family': 'serif', 'serif': ['Computer Modern', 'CMU Serif', 'Times New Roman']})
    # To use proper font for, e.g., Gamma if usetex is set to False
    rc('mathtext', fontset='cm')

    rc('text', usetex=True)
    import matplotlib.pyplot as plt
    plt.rcParams.update({'text.latex.preview': True})

    import pylab as pl

    # I use json to make sure the input is sanitized
    import json

    print_comment = False

    if not all_data.get('use_latex', False):
        rc('text', usetex=False)

    #x = all_data['x']
    #bands = all_data['bands']
    paths = all_data['paths']
    tick_pos = all_data['tick_pos']
    tick_labels = all_data['tick_labels']

    # Option for bands (all, or those of type 1 if there are two spins)
    further_plot_options1 = {}
    further_plot_options1['color'] = all_data.get('bands_color', 'k')
    further_plot_options1['linewidth'] = all_data.get('bands_linewidth', 0.5)
    further_plot_options1['linestyle'] = all_data.get('bands_linestyle', None)
    further_plot_options1['marker'] = all_data.get('bands_marker', None)
    further_plot_options1['markersize'] = all_data.get('bands_markersize', None)
    further_plot_options1['markeredgecolor'] = all_data.get('bands_markeredgecolor', None)
    further_plot_options1['markeredgewidth'] = all_data.get('bands_markeredgewidth', None)
    further_plot_options1['markerfacecolor'] = all_data.get('bands_markerfacecolor', None)

    # Options for second-type of bands if present (e.g. spin up vs. spin down)
    further_plot_options2 = {}
    further_plot_options2['color'] = all_data.get('bands_color2', 'r')
    # Use the values of further_plot_options1 by default
    further_plot_options2['linewidth'] = all_data.get('bands_linewidth2',
        further_plot_options1['linewidth']
        )
    further_plot_options2['linestyle'] = all_data.get('bands_linestyle2',
        further_plot_options1['linestyle']
        )
    further_plot_options2['marker'] = all_data.get('bands_marker2',
        further_plot_options1['marker']
        )
    further_plot_options2['markersize'] = all_data.get('bands_markersize2',
        further_plot_options1['markersize']
        )
    further_plot_options2['markeredgecolor'] = all_data.get('bands_markeredgecolor2',
        further_plot_options1['markeredgecolor']
        )
    further_plot_options2['markeredgewidth'] = all_data.get('bands_markeredgewidth2',
        further_plot_options1['markeredgewidth']
        )
    further_plot_options2['markerfacecolor'] = all_data.get('bands_markerfacecolor2',
        further_plot_options1['markerfacecolor']
        )

    fig = pl.figure()
    p = fig.add_subplot(1,1,1)

    first_band_1 = True
    first_band_2 = True

    for path in paths:
        if path['length'] <= 1:
            # Avoid printing empty lines
            continue
        x = path['x']
        #for band in bands:
        for band, band_type in zip(path['values'], all_data['band_type_idx']):

            # For now we support only two colors
            if band_type % 2 == 0:
                further_plot_options = further_plot_options1
            else:
                further_plot_options = further_plot_options2

            # Put the legend text only once
            label = None
            if first_band_1 and band_type % 2 == 0:
                first_band_1 = False
                label = all_data.get('legend_text', None)
            elif first_band_2 and band_type % 2 == 1:
                first_band_2 = False
                label = all_data.get('legend_text2', None)

            p.plot(x, band, label=label,
                   **further_plot_options
            )


    p.set_xticks(tick_pos)
    p.set_xticklabels(tick_labels)
    p.set_xlim([all_data['x_min_lim'], all_data['x_max_lim']])
    p.set_ylim([all_data['y_min_lim'], all_data['y_max_lim']])
    p.xaxis.grid(True, which='major', color='#888888', linestyle='-', linewidth=0.5)

    if all_data.get('plot_zero_axis', False):
        p.axhline(
            0.,
            color=all_data.get('zero_axis_color', '#888888'),
            linestyle=all_data.get('zero_axis_linestyle', '--'),
            linewidth=all_data.get('zero_axis_linewidth', 0.5),
            )
    if all_data['title']:
        p.set_title(all_data['title'])
    if all_data['legend_text']:
        p.legend(loc='best')
    p.set_ylabel(all_data['yaxis_label'])

    try:
        if print_comment:
            print all_data['comment']
    except KeyError:
        pass
    pl.close()
    #pl.show()
    return fig

In [19]:
@capture(GET_INFO_ERR)
def get_workchain_info(pk):
    from mc_vasp_example import vasp_example as vex
    vex.load_dbenv_if_not_loaded()
    from aiida.orm import load_node, Code
    from aiida.utils.timezone import now
    
    state = 'N/A'
    wc = load_node(pk)
    
    end_time = now()
    try:
        if wc.has_finished():
            state = 'Finished'
            end_time = wc.mtime
            if wc.has_finished_ok():
                state = 'Finished OK'
            elif wc.has_failed():
                state = 'Failed'
            elif wc.has_aborted():
                state = 'Aborted'
        else:
            state = 'Running'
    except AttributeError as err:
        raise TypeError('{} Is not a WorkCalculation'.format(pk))
    
    if state != 'N/A':
        runtime_delta = end_time - wc.ctime
        runtime = runtime_delta.total_seconds()
    else:
        runtime = 'N/A'
    
    inputs = wc.get_inputs_dict()
    
    structure = inputs.get('structure', None)
    formula = ''
    if structure:
        formula = wc.inp.structure.get_formula()
    
    codename = inputs.get('vasp_codename', None)
    computer = ''
    if codename:
        computer = Code.get_from_string(codename.value).get_computer().name
        
    outputs = wc.get_outputs_dict()
    
    out_bands = outputs.get('bands', None)
    bands_data = None
    if out_bands:
        bands_data = out_bands._matplotlib_get_dict()
    
    results = 'N/A'
    if out_bands:
        bands_calc_res = out_bands.inp.output_band.get_outputs_dict().get('output_parameters', None)
        results = dict2html(bands_calc_res.get_dict())
    
    return {'state': state, 'runtime': runtime, 'formula': formula, 'computer': computer, 'bands': bands_data, 'results': results}


def get_workchain_tree(pk, verdi_bin='verdi'):
    import subprocess as sp
    output = sp.check_output([verdi_bin, 'work', 'tree', str(pk)])
    TREE.clear_output()
    with TREE:
    #return sp.check_output([verdi_bin, 'work', 'tree', str(pk)])
        print output
   

def get_verdi_work_list(pk, verdi_bin='verdi'):
    import subprocess as sp
    output = sp.check_output([verdi_bin, 'work', 'list'])
    WORK_LIST.clear_output()
    with WORK_LIST:
        print output
        

def get_verdi_calc_list(pk, verdi_bin='verdi'):
    import subprocess as sp
    output = sp.check_output([verdi_bin, 'calculation', 'list'])
    CALC_LIST.clear_output()    
    with CALC_LIST:
        print output

        
def preplot(check_button):
    from matplotlib import pyplot
    
    #global WORKCHAIN_PK
    #info = get_workchain_info(WORKCHAIN_PK.value)
    #bands = info.get('bands', 'NA')
    #if bands:
    #    display(plot_mpl(bands))
        
    pyplot.close()
    

def update_bands(bands):
    global BANDS, OUTPUT
    fig = plot_mpl(bands)
    BANDS = ipw.Output()
    output_children = [i for i in OUTPUT.children]
    output_children[1] = BANDS
    OUTPUT.children = output_children
    BANDS.append_display_data(fig)
    

def check_workchain(check_button):
    global WORKCHAIN_PK, VERDI_BIN, STATE, RUNTIME, TREE, RESULTS
    
    check_button.disabled=True
    try:
        info = get_workchain_info(WORKCHAIN_PK.value)
        #state, runtime, formula, computer = get_workchain_info(WORKCHAIN_PK.value)
        STATE.value = info.get('state', 'N/A')
        RUNTIME.value = '{} sec'.format(info.get('runtime', 'N/A'))
        FORMULA.value = info.get('formula', 'N/A')
        COMPUTER.value = info.get('computer', 'N/A')
        RESULTS.value = info.get('results', 'N/A')
        #TREE.value = get_workchain_tree(WORKCHAIN_PK.value, verdi_bin=VERDI_BIN.value)
        get_workchain_tree(WORKCHAIN_PK.value, verdi_bin=VERDI_BIN.value)
        bands = info.get('bands', None)
        BANDS.clear_output()
        if bands:
            update_bands(bands)
            
        get_verdi_work_list(WORKCHAIN_PK.value, verdi_bin=VERDI_BIN.value)
        get_verdi_calc_list(WORKCHAIN_PK.value, verdi_bin=VERDI_BIN.value)
    finally:
        check_button.disabled = False

    
#CHECK.on_click(preplot)
CHECK.on_click(check_workchain)

In [20]:
INPUT_STATE = ipw.VBox([WORKCHAIN_PK, COMPUTER, STATE, RUNTIME, FORMULA])
TOP = ipw.HBox([INPUT_STATE, TREE])
LISTS = ipw.Accordion([ipw.VBox([WORK_LIST, ipw.Label(value=''), CALC_LIST])])
LISTS.set_title(0, 'Running Processes')
LISTS.selected_index = None
BOTTOM = ipw.VBox([OUTPUT, LISTS])
display(ipw.VBox([
    TOP,
    BOTTOM,
    CHECK,
    GET_INFO_ERR
]))

VBox(children=(HBox(children=(VBox(children=(IntText(value=0, description=u'Workchain'), HTML(value=u'', description=u'Computer'), HTML(value=u'', description=u'State'), HTML(value=u'', description=u'Runtime'), HTML(value=u'', description=u'Structure'))), Output())), VBox(children=(HBox(children=(HTML(value=u'', description=u'Results'), Output())), Accordion(children=(VBox(children=(Output(), Label(value=u''), Output())),), selected_index=None, _titles={u'0': 'Running Processes'}))), Button(description=u'Check ', style=ButtonStyle()), Output()))

In [21]:
#!/Users/ricohaeuselmann/.venvwrap/matcloud-jupyter-py2/bin/verdi work list

In [22]:
#!/Users/ricohaeuselmann/.venvwrap/matcloud-jupyter-py2/bin/verdi calculation list

In [23]:
## Output: Add Total energy + bandstructure

In [24]:
DEV_NULL = ipw.Output()
from matplotlib import pyplot
with DEV_NULL:
    fig = plot_mpl(get_workchain_info(813)['bands'])
    display(fig)
    pyplot.close()

In [25]:
#Si - 813
#GaAs - 859